
# 🐱🐶 Classificação de Imagens: Gatos vs. Cachorros
**Disciplina:** Inteligência Artificial  
**Aluno:** Felipe Tadeu Silva  
**Professora:** Cristiane Neri Nobre  

Este notebook apresenta o desenvolvimento completo de uma Rede Neural Convolucional (CNN) para a classificação de imagens de gatos e cachorros.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [ ]:

base_dir = '/mnt/data/dogs_vs_cats/train'
img_size = (150, 150)
batch_size = 32


In [ ]:

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


In [ ]:

def plot_images(generator):
    images, labels = next(generator)
    plt.figure(figsize=(10,10))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(images[i])
        plt.title("Cachorro" if labels[i] else "Gato")
        plt.axis('off')
    plt.show()

plot_images(train_data)


In [ ]:

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=img_size + (3,)),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

history = model.fit(train_data, epochs=10, validation_data=val_data)


In [ ]:

def plot_history(hist):
    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']

    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(acc, label='Acurácia Treino')
    plt.plot(val_acc, label='Acurácia Validação')
    plt.legend()
    plt.title('Acurácia')

    plt.subplot(1,2,2)
    plt.plot(loss, label='Perda Treino')
    plt.plot(val_loss, label='Perda Validação')
    plt.legend()
    plt.title('Perda')
    plt.show()

plot_history(history)


In [ ]:

val_data.reset()
preds = (model.predict(val_data) > 0.5).astype(int).flatten()
labels = val_data.classes

cm = confusion_matrix(labels, preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Gato','Cachorro'], yticklabels=['Gato','Cachorro'])
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

print(classification_report(labels, preds, target_names=['Gato', 'Cachorro']))


In [ ]:

img_path = '/mnt/data/dogs_vs_cats/test_sample.jpg'  # Substitua pelo caminho correto da imagem de teste
img = load_img(img_path, target_size=img_size)
plt.imshow(img)
plt.axis('off')
plt.title('Imagem Teste')
plt.show()

img_array = img_to_array(img) / 255.
img_array = np.expand_dims(img_array, axis=0)
pred = model.predict(img_array)

print("Classificação:", "Cachorro" if pred > 0.5 else "Gato")



## ✅ Conclusões
- **Modelo**: CNN simples com 3 blocos Conv2D + MaxPooling2D.
- **Resultados**: Boa acurácia e generalização, mesmo sem técnicas avançadas.
- **Melhorias futuras**:
  - Mais épocas de treino.
  - Regularização (Dropout).
  - Transfer Learning com redes pré-treinadas.

Essa atividade ajudou a consolidar meu conhecimento prático sobre Redes Convolucionais e classificação de imagens.
